In [1]:
import pymongo  #version:3.4.0
from pymongo import MongoClient
client = pymongo.MongoClient()
# from pymongo import Connection
# connection = Connection()#這是在自己的本機建的，所以用connection

In [3]:
db = client.Candidate
db.collection_names()

[u'comments', u'posts', u'user']

In [4]:
# facebook_id = '46251501064' #Tsai
# facebook_page_name = 'Tsai'
facebook_id = '136845026417486' #Ke
facebook_page_name = 'Ke'
# facebook_id = '188311137478' #ChinaTimes 
# facebook_page_name = 'ChinaTimes'
# facebook_id = '241284961029'
# facebook_page_name = 'udn' 
# facebook_id = '394896373929368' 
# facebook_page_name = 'ltn' 
# facebook_id = '232633627068' 
# facebook_page_name = 'AppleDaily'

In [7]:
# 匯入在2015-2017年間留言多於一則以上的使用者名單
user_list = []
file = open('user_Ke.txt', 'r') 
userfile = file.read()
# print userfile.split(',')[0]
user_list.append(userfile.split(','))
print user_list[0][0:20]
print len(user_list[0])

['10152022265951074', '1759351094302693', '10210364671662480', '1326806630681931', '1589009181115633', '1077495905651938', '1126957180742085', '10155759176554392', '1344969395518603', '1449199925114234', '1275073169177386', '10208403068808846', '1884489851831932', '1921928434706238', '10206288992902187', '776034192572665', '826486847374112', '1230572770395572', '1214268621930050', '1420373134645225']
37173


## 一篇一篇標記

In [9]:
import string
import json

ke_sup=['柯P辛苦了','柯p辛苦了','柯Ｐ辛苦了','醫龍國','台北有你真好','柯大'
        '柯P好棒','柯p好棒','柯Ｐ好棒','柯P最棒','柯P讚','柯p讚','柯Ｐ讚','市長好棒',
        '謝謝市長','謝謝柯P','謝謝柯p','謝謝柯Ｐ','謝謝柯市長','支持柯','支持市長','支持你連任','連任市長','我們支持你',
        '市長連任','連任成功','台北市需要你','柯神','柯阿伯','好市長','愛民如己',
        '市長很棒','雙北市長','總統大位','可以選總統嗎拜託','選總統啦','把你送進總統府','把你送去總統府','前進總統府',
        '白色力量加油','柯Ｐ加油','柯P加油','柯市長加油','柯p加油','市長加油','加油柯市長','加油柯P','愛柯P','市長加油',
       '市府的用心','辛苦了','市長我愛你','加油！柯市長！','柯市長，加油','很棒的市長',
       '務實的市長','👍','生日快樂']
ke_not_sup=['白痴市長','白癡市長','柯屁','嘴砲','眼高手低','土皇帝','柯皇','柯文哲下臺','柯文哲下台',
            '垃圾不分藍綠','柯文哲不要臉','白目的力量','最爛台北市長']

cannot = []
user_dict={}

for u in user_list[0]:
    for c in db.comments.find({'user_id':u,'page_name':'Ke','year':{'$gte':2015},'year':{'$lte':2017}}):
        if any (ts in c['comment'].encode('utf-8').translate(None, string.punctuation) for ts in ke_sup):
    #         把user_id存起來
            if not user_dict.has_key(c['user_id']):
                user_dict[c['user_id']]='1'
            else:
                user_dict[c['user_id']]+=',1'
    #                 print c['comment_id']#c['comment]為unicode 
            db.detector_one_by_one.insert({'comment':c['comment'],
                                    'user_id':c['user_id'],
                                    'user_name':c['user_name'],
                                    'comment_id':c['comment_id'],
                                    'post_id':c['post_id'],
                                    'page_name':facebook_page_name,
                                    'page_id':facebook_id,
                                    'comment_count':c['comment_count'],
                                    'like_count':c['like_count'],
                                    'stand':1})
        elif any (no in c['comment'].encode('utf-8').translate(None, string.punctuation) for no in ke_not_sup):
                if not user_dict.has_key(c['user_id']):
                    user_dict[c['user_id']]='-1'
                else:
                    user_dict[c['user_id']]+=',-1'

                db.detector_one_by_one.insert({'comment':c['comment'],
                                        'user_id':c['user_id'],
                                        'user_name':c['user_name'],
                                        'comment_id':c['comment_id'],
                                        'post_id':c['post_id'],
                                        'page_name':facebook_page_name,
                                        'page_id':facebook_id,
                                        'comment_count':c['comment_count'],
                                        'like_count':c['like_count'],
                                        'stand':-1,
                                        'auto':0})
        else:
            if not user_dict.has_key(c['user_id']):
                user_dict[c['user_id']]='0'
            else:
                user_dict[c['user_id']]+=',0'

            db.detector_one_by_one.insert({'comment':c['comment'],
                            'user_id':c['user_id'],
                            'user_name':c['user_name'],
                            'comment_id':c['comment_id'],
                            'post_id':c['post_id'],
                            'page_name':facebook_page_name,
                            'page_id':facebook_id,
                            'comment_count':c['comment_count'],
                            'like_count':c['like_count'],
                            'stand':0,
                            'auto':0})


db.detector_one_by_one.count()
db.update
# comments.update()
print db.detector_one_by_one.count()
# db.posts.find({})

#把user_dict存成json檔
with open('user_one_by_one.json_ke', 'w') as f:
    json.dump(user_dict, f)

/Users/TingYen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:70: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/Users/TingYen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/Users/TingYen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:53: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


256134


### Finding average of a list: http://stackoverflow.com/questions/9039961/finding-the-average-of-a-list

In [10]:
#找出共有幾位留言者
print len(user_dict)

37172


In [11]:
# 計算這個留言者支持跟不支持的數值，小數位取到第一位
count={}
for u,i in user_dict.items():
#     i = map(int, i.translate(None, string.punctuation))  ###這裡會把負號弄掉！！！！！！
    i = [int(x.strip()) for x in i.split(',')]  #所以改成這樣
    h= float(reduce(lambda q,p: p+q, i)/float(len(i)))
    k = round(h,2)
#     if len(str(h).split('.')[1])>0:
    if not count.has_key(k):
        count[k]=1
    count[k]+=1
#     print u,h #不要印！！！！！！！！

In [12]:
# 印出來看看
for x, c in count.items():
    print x, c

0.5 2565
0.0 24767
0.25 875
1.0 1805
0.64 5
0.38 87
0.06 147
0.49 2
-0.5 59
0.03 105
0.13 277
0.46 6
-0.83 2
0.24 21
0.69 6
0.44 25
-0.18 6
0.4 259
0.42 21
-0.06 10
-0.28 2
0.67 412
0.55 10
0.8 37
-0.95 2
0.65 2
0.71 14
0.96 2
0.19 41
0.56 12
-0.69 2
0.43 77
0.72 2
0.57 33
-0.17 19
0.59 3
0.75 104
0.94 2
0.17 483
0.29 151
-0.67 5
-0.08 10
0.82 3
0.09 147
0.01 39
0.21 40
-0.33 44
0.51 2
0.14 368
0.6 85
0.15 85
0.1 183
0.28 11
0.86 6
-0.16 3
0.76 3
0.26 14
0.62 4
-0.22 6
-0.36 3
-0.01 7
0.11 221
-0.31 3
-0.75 2
0.18 71
0.92 3
0.78 9
0.89 4
-0.43 2
-0.07 7
0.22 97
0.79 2
0.45 12
-0.15 3
-0.6 2
-0.05 15
0.68 2
0.33 1408
-0.21 3
-0.04 16
0.53 5
0.41 12
0.23 36
-0.29 5
0.04 95
-0.12 4
0.02 71
0.63 14
0.39 8
0.34 3
0.87 2
-0.09 14
0.74 2
0.3 47
0.47 10
-0.14 19
-0.19 2
-0.02 8
-0.44 3
0.58 8
0.7 2
0.35 12
-0.2 27
-0.1 7
0.08 215
0.91 2
0.12 41
-0.4 4
0.77 3
0.07 142
0.31 33
0.36 40
-0.11 14
0.54 8
0.16 27
-0.8 2
0.48 6
0.83 21
0.61 3
0.73 7
-0.13 12
-0.23 2
0.88 6
0.9 2
-0.71 2
0.27 47
0.05 1

In [20]:
# 印出來看看那些平均不是0,1或-1的使用者是哪些
user_05 = []
for u,i in user_dict.items():
    i = map(int, i.translate(None, string.punctuation))
    h= float(reduce(lambda q,p: p+q, i)/float(len(i)))
#     if len(str(h).split('.')[1])>0:
#     if round(h,2)!=0.00 and h!=1.00 and h!=-1.00:
#         print u,h
    if round(h,2)>=0.5 and h!=1.00:
        print u, h
        user_05.append(u)

10152022265951074 0.5
1113982298660803 0.5
1759351094302693 0.5
1194015430623297 0.5
1571403983162367 0.666666666667
1130233263666203 0.5
10154091840965826 0.5
10208684265082739 0.5
10207724872893252 0.5
1140596289344116 0.5
1175626229229623 0.5
1317134851634816 0.666666666667
1118149168246954 0.5
773318232685810 0.615384615385
1334746076585302 0.5
10208359800656534 0.5
10206299845438314 0.5
1300707836653221 0.5
1068642276551140 0.5
658666877675263 0.5
10208133499073121 0.5
1139064446152335 0.5
1502864749753899 0.5
1379646352048799 0.5
10207039972092864 0.5
10155132914683900 0.666666666667
877955852332428 0.5
10204301459994564 0.5
10153832849678247 0.75
10154368097769375 0.5
1090124484402081 0.5
10207179976171779 0.5
1584358228259803 0.666666666667
1876575285921687 0.5
1300911799921649 0.714285714286
1627002177609703 0.5
10154966459181530 0.5
10201786586224487 0.5
1358574290826659 0.5
1183602321670486 0.5
10154314418978497 0.5
1869218826667961 0.5
1903000543265350 0.5
1869006310053734 

In [21]:
# 找出某個使用者的留言
for u in user_05:
    for i, c in enumerate(db.detector_one_by_one.find({'user_id':u})):
        print c['comment']

市府的廁所真的不通風😓
市府的廁所真的不通風😓
活在當下，認真過每一天～生日快樂～市長
活在當下，認真過每一天～生日快樂～市長
新年快樂
市長加油
市長，還要開放校園嗎…請您好好想想
市長很棒，加油
柯P加油😤😤😤💪🏻
也太可愛
許子懿吳奕臻超可愛哈哈哈
市長加油！！
柯市長加油！
市長加油加油。
祝身體健康，閤家平安快樂。
我還是很愛柯文哲！
市長我愛你～
最認真の市長〰柯P加油
柯P加油
柯p你這樣好想遷戶口ㄑ台北噢☺️
那些很會講話の人，都沒在做事
整天砰擊對手，浪費納稅人的錢就飽了

雖然柯Pの直腸子總是落人口舌⋯
而且你の高人氣一直招人妒忌
但你の努力大家都看的到👏🏻👏🏻
好羨慕台北市民～有這麽好の市長
柯P加油。市府團隊加油👋🏻✋🏻
希望台灣能再多幾個柯P..⋯加油
如果每個政治人物都像柯p、花媽這樣有在做事的、台灣就會進步了✔️
柯P嗡嗡包好可愛😁
柯P加油～辛苦了
一日北高...太猛! 期待四年後了!
柯P, 台灣需要你的堅持和反省. 辛苦了!
真的風大雨大！傘瞬間全毀滅！！請業者拿出良心不要再為了錢去寵那些不會將心比心的自私客，去看看櫃姐們颱風天下班的危險，狼狽狀況~住遠的還不見得有車可回家、什麼颱風天不出門不消費~百貨沒生意沒人潮沒錢賺就不會開…狗屁話！！…那櫃兄弟姊妹們還要再冒多久的危險啊~才甘願將心比心、每個人都是人生父母養，都是別人的家人，別別人的小孩死不完~
就直接按照政府宣佈的停班~不要營業！（希望政府相關單位好好強制要求服務業百貨業者配合，違者重罰！）
讓大家都能安全無需這樣冒險上下班就好啦~
柯P加油！別跟前輩的一樣…
繼續堅持做對的事.柯P加油!
把事情做好跟做好事～
是兩回事。。柯P加油！！
柯市長，晚安好😊
期待更美好的明天~
因為有你~
😍柯市長早安
陳醫師帶領六龜孩子太👍了
六龜孩子好棒
柯市長真的是有在做事的市長
沒看過這麼萌的市長!!!太可愛 <3 市長加油
柯市長加油!!! 雖然我沒有辦法投給您，但真的很喜歡您關於長照方面的政策!!很有遠見性的規劃
柯P柯P柯P   YA～
我覺得你不適合當市長，比較適合當總統。
加油 柯P加油
柯P加油
這群腦殘粉看到沒？你們柯皇不敢承認⋯
知硬凹不過去⋯手下出來承認啦😂😂😂
http://m.appledaily.com.tw/realtimenews/article/new/2

KeyboardInterrupt: 

## 用shell找mongoDB有重複的
#### https://www.wiredprairie.us/blog/index.php/archives/1857

In [19]:
for c in db.comments.find({'comment_id':'10151280569516065_10850237'}):
    print c['comment']


馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
馬總統邀辯論服貿 這麼懂就出來辯論阿 迴避什麼?
